In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
!pip install snac

In [1]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Qwen3 new models
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # Other very popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/orpheus-3b-0.1-ft",
#     max_seq_length= 2048, # Choose any for long context!
#     dtype = None, # Select None for auto detection
#     load_in_4bit = False, # Select True for 4bit which reduces memory usage
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "canopylabs/orpheus-3b-0.1-pretrained",
    model_name = "dkulemin/lora_orpheus_gothic",
    max_seq_length= 2048,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.4: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
Unsloth 2025.8.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
from datasets import load_dataset
dataset = load_dataset("dkulemin/gothic_orig_ru_dialogues", split="train")

README.md:   0%|          | 0.00/356 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/263M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/251M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5069 [00:00<?, ? examples/s]

In [ ]:
# dataset = dataset.filter(lambda x: x["source"] == "13")

In [ ]:
# dataset = dataset.remove_columns(["source"])

In [ ]:
dataset

Dataset({
    features: ['source', 'text', 'audio'],
    num_rows: 5069
})

In [4]:
from IPython.display import Audio

In [ ]:
dataset[0]

{'text': 'Я... я... я Каин, ученик Кор Галома. Ты ведь слышал о нем, да?',
 'audio': {'path': 'DIA_CAINE_HALLO_13_01.WAV',
  'array': array([3.05175781e-05, 2.44140625e-04, 3.96728516e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
  'sampling_rate': 44100}}

In [ ]:
inputs = tokenizer(dataset[0]['text'], return_tensors="pt")
dataset[0]['text'], inputs

('Я... я... я Каин, ученик Кор Галома. Ты ведь слышал о нем, да?',
 {'input_ids': tensor([[128000,  86491,   1131,  46410,   1131,  46410, 118290,  19479,     11,
          101990,   9882,   4898, 107212,  61565,  16331, 109080,     13, 120076,
          113577, 125942,  16331,   8379, 102461,     11,  92910,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]])})

In [ ]:
Audio(dataset[0]['audio']['array'], rate=dataset[0]['audio']['sampling_rate'])

In [ ]:
Audio(dataset[70]['audio']['array'], rate=dataset[70]['audio']['sampling_rate'])

In [5]:
#@title Tokenization Function

import locale
import torchaudio.transforms as T
import os
import torch
from snac import SNAC
locale.getpreferredencoding = lambda: "UTF-8"
ds_sample_rate = dataset[0]["audio"]["sampling_rate"]

snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.to("cuda")
def tokenise_audio(waveform):
  waveform = torch.from_numpy(waveform).unsqueeze(0)
  waveform = waveform.to(dtype=torch.float32)
  resample_transform = T.Resample(orig_freq=ds_sample_rate, new_freq=24000)
  waveform = resample_transform(waveform)

  waveform = waveform.unsqueeze(0).to("cuda")

  #generate the codes from snac
  with torch.inference_mode():
    codes = snac_model.encode(waveform)

  all_codes = []
  for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))


  return all_codes

def add_codes(example):
    # Always initialize codes_list to None
    codes_list = None

    try:
        answer_audio = example.get("audio")
        # If there's a valid audio array, tokenise it
        if answer_audio and "array" in answer_audio:
            audio_array = answer_audio["array"]
            codes_list = tokenise_audio(audio_array)
    except Exception as e:
        print(f"Skipping row due to error: {e}")
        # Keep codes_list as None if we fail
    example["codes_list"] = codes_list

    return example

dataset = dataset.map(add_codes, remove_columns=["audio"])

tokeniser_length = 128256
start_of_text = 128000
end_of_text = 128009

start_of_speech = tokeniser_length + 1
end_of_speech = tokeniser_length + 2

start_of_human = tokeniser_length + 3
end_of_human = tokeniser_length + 4

start_of_ai = tokeniser_length + 5
end_of_ai =  tokeniser_length + 6
pad_token = tokeniser_length + 7

audio_tokens_start = tokeniser_length + 10

dataset = dataset.filter(lambda x: x["codes_list"] is not None)
dataset = dataset.filter(lambda x: len(x["codes_list"]) > 0)

def remove_duplicate_frames(example):
    vals = example["codes_list"]
    if len(vals) % 7 != 0:
        raise ValueError("Input list length must be divisible by 7")

    result = vals[:7]

    removed_frames = 0

    for i in range(7, len(vals), 7):
        current_first = vals[i]
        previous_first = result[-7]

        if current_first != previous_first:
            result.extend(vals[i:i+7])
        else:
            removed_frames += 1

    example["codes_list"] = result

    return example

dataset = dataset.map(remove_duplicate_frames)

tok_info = '''*** HERE you can modify the text prompt
If you are training a multi-speaker model (e.g., canopylabs/orpheus-3b-0.1-ft),
ensure that the dataset includes a "source" field and format the input accordingly:
- Single-speaker: f"{example['text']}"
- Multi-speaker: f"{example['source']}: {example['text']}"
'''
print(tok_info)

def create_input_ids(example):
    # Determine whether to include the source field
    text_prompt = f"{example['source']}: {example['text']}" if "source" in example else example["text"]

    text_ids = tokenizer.encode(text_prompt, add_special_tokens=True)
    text_ids.append(end_of_text)

    example["text_tokens"] = text_ids
    input_ids = (
        [start_of_human]
        + example["text_tokens"]
        + [end_of_human]
        + [start_of_ai]
        + [start_of_speech]
        + example["codes_list"]
        + [end_of_speech]
        + [end_of_ai]
    )
    example["input_ids"] = input_ids
    example["labels"] = input_ids
    example["attention_mask"] = [1] * len(input_ids)

    return example


dataset = dataset.map(create_input_ids, remove_columns=["text", "codes_list"])
columns_to_keep = ["input_ids", "labels", "attention_mask"]
columns_to_remove = [col for col in dataset.column_names if col not in columns_to_keep]

dataset = dataset.remove_columns(columns_to_remove)

config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/79.5M [00:00<?, ?B/s]

Parameter 'function'=<function add_codes at 0x7cf75e5c2ca0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5069 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5069 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5069 [00:00<?, ? examples/s]

Map:   0%|          | 0/5069 [00:00<?, ? examples/s]

*** HERE you can modify the text prompt
If you are training a multi-speaker model (e.g., canopylabs/orpheus-3b-0.1-ft),
ensure that the dataset includes a "source" field and format the input accordingly:
- Single-speaker: f"{example['text']}"
- Multi-speaker: f"{example['source']}: {example['text']}"



Map:   0%|          | 0/5069 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments,Trainer,DataCollatorForSeq2Seq
# from unsloth import is_bfloat16_supported
trainer = Trainer(
    model = model,
    train_dataset = dataset,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        # max_steps = None,
        # num_train_epochs = 10,
        # fp16 = not is_bfloat16_supported(),
        # bf16 = is_bfloat16_supported(),
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [7]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.873 GB of memory reserved.


In [34]:
# 4 epoch

In [35]:
FastLanguageModel.for_training(model)
None

In [36]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,069 | Num Epochs = 1 | Total steps = 1,268
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 97,255,424 of 3,398,122,496 (2.86% trained)


Step,Training Loss
1,4.404100
2,4.792600
3,5.072800
4,4.604100
5,4.372500
6,3.792300
7,4.577100
8,3.996800
9,4.687800
10,4.114100


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4313.3759 seconds used for training.
71.89 minutes used for training.
Peak reserved memory = 5.383 GB.
Peak reserved memory for training = 0.51 GB.
Peak reserved memory % of max memory = 36.517 %.
Peak reserved memory for training % of max memory = 3.46 %.


In [17]:
from google.colab import drive, userdata
# drive.mount('/content/gdrive/')

In [18]:
token = userdata.get('HF_TOKEN')

In [37]:
model.push_to_hub("dkulemin/lora_orpheus_gothic", token = token) # Online saving
tokenizer.push_to_hub("dkulemin/lora_orpheus_gothic", token = token) # Online saving

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

Saved model to https://huggingface.co/dkulemin/lora_orpheus_gothic


No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
# model.save_pretrained("gdrive/My Drive/GOTHIC/models/orpheus/lora_orpheus_full")
# tokenizer.save_pretrained("gdrive/My Drive/GOTHIC/models/orpheus/lora_orpheus_full")

('gdrive/My Drive/GOTHIC/models/orpheus/lora_cain_13/tokenizer_config.json',
 'gdrive/My Drive/GOTHIC/models/orpheus/lora_cain_13/special_tokens_map.json',
 'gdrive/My Drive/GOTHIC/models/orpheus/lora_cain_13/chat_template.jinja',
 'gdrive/My Drive/GOTHIC/models/orpheus/lora_cain_13/tokenizer.json')

In [ ]:
# token = userdata.get('HF_TOKEN')
# model.push_to_hub_merged('dkulemin/orpheus_cain_13_16bit', tokenizer, save_method = "merged_16bit", token=token)
# model.push_to_hub_merged('dkulemin/orpheus_cain_13_4bit', tokenizer, save_method = "merged_4bit", token=token)


RuntimeError: Unsloth: Merging into 4bit will cause your model to lose accuracy if you plan
to merge to GGUF or others later on. I suggest you to do this as a final step
if you're planning to do multiple saves.
If you are certain, change `save_method` to `merged_4bit_forced`.

In [2]:
import locale
import torchaudio.transforms as T
import os
import torch
from snac import SNAC
locale.getpreferredencoding = lambda: "UTF-8"

snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.to("cuda")

In [3]:
prompts = [
    "Привет, человек",
    # Пачо
    # "Info_Pacho_GRD_FM_13_01"
    "Ну, сколько можно тебя ждать?",
    # "Info_Pacho_GRD_FM_13_03"
    "Ладно, все равно он сам сильно задерживается.",
    # # "Info_Pacho_GRD_FM_END_13_01"
    "А куда это ты собрался?",
    # # "Info_Pacho_GRD_FM_END_13_03"
    "Эй, парни, идите сюда! У нас предатель!",

    # # "Info_Org_867_Raeuber_BDT_13_00"
    # "Я получаю достаточно, как стражник лагеря.",
    # # "Org_867_Raeuber_About_13_00"
    # "Квентин убит.",
    # # "Org_867_Raeuber_About_13_01"
    # "Скорее всего его убили стражники из Старого лагеря.",
    # # "Org_867_Raeuber_About_13_02"
    # "Франко наш новый руководитель.",
    # # "Org_867_Raeuber_About_13_03"
    # "Все, как обычно.",
    # # "Org_867_Raeuber_Banditencamp_13_01"
    # "Это лагерь бандитов, парень.",
    # # "Org_867_Raeuber_GUARD_Info_13_00"
    # "Стой!",
    # # "Org_867_Raeuber_GUARD_Info_13_02"
    # "Тебе здесь нечего делать, парень.",
    # # "Org_867_Raeuber_GoAway_Info_13_00"
    # "Еще один придурок из Старого лагеря.",
    # # "Org_867_Raeuber_Main_13_00"
    # "Сейчас нас возглавляет Франко.",
    # # "Org_867_Raeuber_Main_13_01"
    # "Квентин главный в лагере.",
    # # "Org_867_Raeuber_Organisators_13_01"
    # "Больше нет.",
    # # "Org_867_Raeuber_Organisators_13_02"
    # "Магам воды и наемникам не нравилось, что мы совершаем набеги на конвои из Старого лагеря.",
    # # "Org_867_Raeuber_Organisators_13_03"
    # "Ларес предложил части воров уйти из Нового лагеря и посроить свой лагерь здесь.",
    # # "Org_867_Raeuber_Organisators_13_04"
    # "Недавно мы напали еще на один отряд стражников.",
    # # "Org_867_Raeuber_Organisators_13_05"
    # "В лагере. Пройди дальше через ущелье.",
    # # "Org_867_Raeuber_Organisators_13_06"
    # "Еще трое крутят болотник в пещере около Нового лагеря.",
    # # "Org_867_Raeuber_WereIsMain_13_00"
    # "Он в лагере. Просто иди дальше через ущелье.",
    # # "Org_867_Raeuber_WereIsMain_13_01"
    # "Он куда-то ушел из лагеря пару дней назад.",

    # # Франко
    # # "Info_Org_869_Raeuber_BDT_13_00"
    # "А ты что думал? Он же глава банды.",
    # # "Info_Org_869_Raeuber_TRADE_13_00"
    # "Да, у меня есть куча товара с нашего склада.",

    # # "SVM_13_YouAskedForIt"
    # "Ты сам напросился!",
]

chosen_voice = "15" # None for single-speaker

In [4]:
#@title Run Inference


FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Moving snac_model cuda to cpu
snac_model.to("cpu")

prompts_ = [(f"{chosen_voice}: " + p) if chosen_voice else p for p in prompts]

all_input_ids = []

for prompt in prompts_:
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  all_input_ids.append(input_ids)

start_token = torch.tensor([[ 128259]], dtype=torch.int64) # Start of human
end_tokens = torch.tensor([[128009, 128260]], dtype=torch.int64) # End of text, End of human

all_modified_input_ids = []
for input_ids in all_input_ids:
  modified_input_ids = torch.cat([start_token, input_ids, end_tokens], dim=1) # SOH SOT Text EOT EOH
  all_modified_input_ids.append(modified_input_ids)

all_padded_tensors = []
all_attention_masks = []
max_length = max([modified_input_ids.shape[1] for modified_input_ids in all_modified_input_ids])
for modified_input_ids in all_modified_input_ids:
  padding = max_length - modified_input_ids.shape[1]
  padded_tensor = torch.cat([torch.full((1, padding), 128263, dtype=torch.int64), modified_input_ids], dim=1)
  attention_mask = torch.cat([torch.zeros((1, padding), dtype=torch.int64), torch.ones((1, modified_input_ids.shape[1]), dtype=torch.int64)], dim=1)
  all_padded_tensors.append(padded_tensor)
  all_attention_masks.append(attention_mask)

all_padded_tensors = torch.cat(all_padded_tensors, dim=0)
all_attention_masks = torch.cat(all_attention_masks, dim=0)

input_ids = all_padded_tensors.to("cuda")
attention_mask = all_attention_masks.to("cuda")
generated_ids = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_new_tokens=1200,
      do_sample=True,
      temperature=0.6,
      top_p=0.95,
      repetition_penalty=1.1,
      num_return_sequences=1,
      eos_token_id=128258,
     use_cache = True
  )
token_to_find = 128257
token_to_remove = 128258

token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)

if len(token_indices[1]) > 0:
    last_occurrence_idx = token_indices[1][-1].item()
    cropped_tensor = generated_ids[:, last_occurrence_idx+1:]
else:
    cropped_tensor = generated_ids

mask = cropped_tensor != token_to_remove

processed_rows = []

for row in cropped_tensor:
    masked_row = row[row != token_to_remove]
    processed_rows.append(masked_row)

code_lists = []

for row in processed_rows:
    row_length = row.size(0)
    new_length = (row_length // 7) * 7
    trimmed_row = row[:new_length]
    trimmed_row = [t - 128266 for t in trimmed_row if t - 128266 > 0]
    code_lists.append(trimmed_row)


def redistribute_codes(code_list):
  layer_1 = []
  layer_2 = []
  layer_3 = []
  for i in range((len(code_list)+1)//7):
    layer_1.append(code_list[7*i])
    layer_2.append(code_list[7*i+1]-4096)
    layer_3.append(code_list[7*i+2]-(2*4096))
    layer_3.append(code_list[7*i+3]-(3*4096))
    layer_2.append(code_list[7*i+4]-(4*4096))
    layer_3.append(code_list[7*i+5]-(5*4096))
    layer_3.append(code_list[7*i+6]-(6*4096))
  codes = [torch.tensor(layer_1).unsqueeze(0),
         torch.tensor(layer_2).unsqueeze(0),
         torch.tensor(layer_3).unsqueeze(0)]

  # codes = [c.to("cuda") for c in codes]
  audio_hat = snac_model.decode(codes)
  return audio_hat

my_samples = []
for code_list in code_lists:
  samples = redistribute_codes(code_list)
  my_samples.append(samples)
from IPython.display import display, Audio
if len(prompts) != len(my_samples):
  raise Exception("Number of prompts and samples do not match")
else:
  for i in range(len(my_samples)):
    print(prompts[i])
    samples = my_samples[i]
    display(Audio(samples.detach().squeeze().to("cpu").numpy(), rate=24000))
# Clean up to save RAM
del my_samples,samples

Привет, человек


Ну, сколько можно тебя ждать?


Ладно, все равно он сам сильно задерживается.


А куда это ты собрался?


Эй, парни, идите сюда! У нас предатель!
